# Sukurkite klasifikavimo modelį: Skanūs Azijos ir Indijos patiekalai


## Virtuvės klasifikatoriai 2

Šioje antroje klasifikavimo pamokoje nagrinėsime `daugiau būdų` klasifikuoti kategorinius duomenis. Taip pat sužinosime apie pasekmes, susijusias su vieno klasifikatoriaus pasirinkimu vietoj kito.

### [**Išankstinis testas**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Būtinos žinios**

Darome prielaidą, kad jau baigėte ankstesnes pamokas, nes naudosime kai kurias anksčiau išmoktas sąvokas.

Šiai pamokai reikės šių paketų:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) yra [R paketų rinkinys](https://www.tidyverse.org/packages), sukurtas tam, kad duomenų mokslas būtų greitesnis, paprastesnis ir įdomesnis!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) sistema yra [paketų rinkinys](https://www.tidymodels.org/packages/) modeliavimui ir mašininio mokymosi užduotims.

-   `themis`: [themis paketas](https://themis.tidymodels.org/) siūlo papildomus receptų žingsnius, skirtus spręsti nesubalansuotų duomenų problemas.

Galite juos įdiegti naudodami šią komandą:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Arba galite naudoti žemiau pateiktą scenarijų, kuris patikrina, ar turite reikalingus paketus šiam moduliui užbaigti, ir, jei jų trūksta, įdiegia juos už jus.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Klasifikacijos žemėlapis**

Mūsų [ankstesnėje pamokoje](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) bandėme atsakyti į klausimą: kaip pasirinkti tarp kelių modelių? Didžiąja dalimi tai priklauso nuo duomenų savybių ir problemos tipo, kurią norime išspręsti (pavyzdžiui, klasifikacija ar regresija?).

Anksčiau sužinojome apie įvairias galimybes, kurias turite klasifikuodami duomenis, naudodami „Microsoft“ parengtą atmintinę. Python mašininio mokymosi sistema, Scikit-learn, siūlo panašią, bet detalesnę atmintinę, kuri gali dar labiau padėti susiaurinti jūsų pasirinkimą tarp vertintojų (kitas terminas klasifikatoriams):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Patarimas: [aplankykite šį žemėlapį internete](https://scikit-learn.org/stable/tutorial/machine_learning_map/) ir sekite kelią, kad perskaitytumėte dokumentaciją.
>
> [Tidymodels nuorodų svetainė](https://www.tidymodels.org/find/parsnip/#models) taip pat pateikia puikią dokumentaciją apie skirtingus modelių tipus.

### **Planas** 🗺️

Šis žemėlapis yra labai naudingas, kai gerai suprantate savo duomenis, nes galite „eiti“ jo keliais iki sprendimo:

-   Turime \>50 pavyzdžių

-   Norime prognozuoti kategoriją

-   Turime pažymėtus duomenis

-   Turime mažiau nei 100K pavyzdžių

-   ✨ Galime pasirinkti Linear SVC

-   Jei tai neveikia, kadangi turime skaitinius duomenis

    -   Galime pabandyti ✨ KNeighbors Classifier

        -   Jei tai neveikia, pabandykite ✨ SVC ir ✨ Ensemble Classifiers

Tai labai naudingas kelias, kurį verta sekti. Dabar pereikime tiesiai prie darbo, naudodami [tidymodels](https://www.tidymodels.org/) modeliavimo sistemą: nuoseklią ir lankstią R paketų kolekciją, sukurtą skatinti gerą statistinę praktiką 😊.

## 2. Padalykite duomenis ir spręskite nesubalansuoto duomenų rinkinio problemą.

Iš ankstesnių pamokų sužinojome, kad mūsų virtuvėse buvo bendrų ingredientų rinkinys. Taip pat pastebėjome, kad virtuvių skaičiaus pasiskirstymas buvo gana netolygus.

Šias problemas spręsime:

-   Pašalindami dažniausiai pasitaikančius ingredientus, kurie sukelia painiavą tarp skirtingų virtuvių, naudodami `dplyr::select()`.

-   Naudodami `recipe`, kuris iš anksto apdoroja duomenis, kad jie būtų paruošti modeliui, taikydami `over-sampling` algoritmą.

Apie tai jau kalbėjome ankstesnėje pamokoje, todėl tai turėtų būti paprasta 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Darbas su nesubalansuotais duomenimis

Nesubalansuoti duomenys dažnai neigiamai veikia modelio veikimą. Daugelis modelių geriausiai veikia, kai stebėjimų skaičius yra vienodas, todėl jie dažnai susiduria su sunkumais dirbant su nesubalansuotais duomenimis.

Yra du pagrindiniai būdai, kaip spręsti nesubalansuotų duomenų rinkinių problemą:

-   pridėti stebėjimų prie mažumos klasės: `Perėmimas` (angl. Over-sampling), pvz., naudojant SMOTE algoritmą, kuris sintetiškai generuoja naujus mažumos klasės pavyzdžius, naudodamas artimiausius šių atvejų kaimynus.

-   pašalinti stebėjimus iš daugumos klasės: `Perėmimas` (angl. Under-sampling)

Ankstesnėje pamokoje mes pademonstravome, kaip spręsti nesubalansuotų duomenų rinkinių problemą naudojant `receptą`. Receptą galima laikyti planu, kuris aprašo, kokie žingsniai turėtų būti taikomi duomenų rinkiniui, kad jis būtų paruoštas duomenų analizei. Mūsų atveju norime užtikrinti vienodą mūsų `mokymo rinkinio` virtuvių skaičiaus pasiskirstymą. Pradėkime!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Dabar esame pasiruošę mokyti modelius 👩‍💻👨‍💻!

## 3. Daugiau nei multinominiai regresijos modeliai

Ankstesnėje pamokoje aptarėme multinominius regresijos modelius. Pažvelkime į lankstesnius klasifikavimo modelius.

### Atraminių vektorių mašinos

Klasifikavimo kontekste `Atraminių vektorių mašinos` yra mašininio mokymosi metodas, kuris siekia rasti *hiperplokštumą*, kuri "geriausiai" atskiria klases. Pažvelkime į paprastą pavyzdį:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ nesiskiria klasės. H2~ skiria, bet tik su mažu tarpu. H3~ skiria maksimaliai.

#### Linijinis atraminių vektorių klasifikatorius

Atraminių vektorių klasterizavimas (SVC) yra atraminių vektorių mašinų šeimos ML technikų dalis. SVC atveju hiperlėktuvą pasirenkama taip, kad jis teisingai atskirtų `daugumą` mokymo stebėjimų, tačiau `gali neteisingai klasifikuoti` kelis stebėjimus. Leidžiant kai kuriems taškams būti neteisingoje pusėje, SVM tampa atsparesnis išskirtims, todėl geriau apibendrina naujus duomenis. Parametras, reguliuojantis šį pažeidimą, vadinamas `kaina`, kurios numatytoji vertė yra 1 (žr. `help("svm_poly")`).

Sukurkime linijinį SVC nustatydami `degree = 1` polinominio SVM modelyje.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Dabar, kai apibrėžėme išankstinio apdorojimo veiksmus ir modelio specifikaciją *darbo eigoje*, galime pereiti prie linijinio SVC mokymo ir rezultatų vertinimo tuo pačiu metu. Kalbant apie našumo rodiklius, sukurkime rodiklių rinkinį, kuris įvertins: `tikslumą`, `jautrumą`, `teigiamą prognozuojamą vertę` ir `F matą`.

> `augment()` pridės stulpelį(-ius) su prognozėmis prie pateiktų duomenų.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Atramos vektorių mašina

Atramos vektorių mašina (SVM) yra atramos vektorių klasifikatoriaus išplėtimas, leidžiantis pritaikyti netiesinę ribą tarp klasių. Iš esmės, SVM naudoja *branduolio triuką* (angl. *kernel trick*), kad išplėstų požymių erdvę ir prisitaikytų prie netiesinių ryšių tarp klasių. Viena populiari ir itin lanksti branduolio funkcija, kurią naudoja SVM, yra *radialinės bazės funkcija*. Pažiūrėkime, kaip ji veiks su mūsų duomenimis.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Daug geriau 🤩!

> ✅ Prašome peržiūrėti:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> norint sužinoti daugiau.

### Artimiausių kaimynų klasifikatoriai

*K*-artimiausių kaimynų (KNN) algoritmas yra metodas, kuriame kiekviena stebėsena prognozuojama remiantis jos *panašumu* su kitomis stebėsenomis.

Pritaikykime jį mūsų duomenims.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Atrodo, kad šis modelis neveikia taip gerai. Tikėtina, kad pakeitus modelio parametrus (žr. `help("nearest_neighbor")`), modelio veikimas pagerės. Būtinai išbandykite.

> ✅ Prašome peržiūrėti:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> kad sužinotumėte daugiau apie *K*-artimiausių kaimynų klasifikatorius.

### Ansambliniai klasifikatoriai

Ansambliniai algoritmai veikia sujungdami kelis bazinius vertintojus, kad sukurtų optimalų modelį, tai daroma:

`bagging`: taikant *vidurkinimo funkciją* bazinei modelių kolekcijai

`boosting`: kuriant modelių seką, kurioje kiekvienas modelis remiasi ankstesniu, siekiant pagerinti prognozavimo tikslumą.

Pradėkime nuo Random Forest modelio, kuris sukuria didelę sprendimų medžių kolekciją ir tada taiko vidurkinimo funkciją, kad sukurtų geresnį bendrą modelį.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Puikus darbas 👏!

Išbandykime ir Boosted Tree modelį.

Boosted Tree apibrėžia ansamblio metodą, kuris sukuria seką nuoseklių sprendimų medžių, kur kiekvienas medis priklauso nuo ankstesnių medžių rezultatų, siekiant palaipsniui sumažinti klaidą. Jis sutelkia dėmesį į neteisingai klasifikuotų elementų svorius ir koreguoja kitą klasifikatorių, kad ištaisytų klaidas.

Yra įvairių būdų, kaip pritaikyti šį modelį (žr. `help("boost_tree")`). Šiame pavyzdyje Boosted trees bus pritaikytas naudojant `xgboost` variklį.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

✅ Prašome peržiūrėti:

-   [Mašininis mokymasis socialiniams mokslininkams](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)

-   [Praktinis mašininis mokymasis su R](https://bradleyboehmke.github.io/HOML/)

-   [Statistinio mokymosi įvadas su R programos taikymu](https://www.statlearning.com/)

-   <https://algotech.netlify.app/blog/xgboost/> - Aptariamas AdaBoost modelis, kuris yra gera alternatyva xgboost.

Norėdami sužinoti daugiau apie ansamblių klasifikatorius.

## 4. Papildoma - kelių modelių palyginimas

Šiame laboratoriniame darbe pritaikėme nemažai modelių 🙌. Gali tapti varginančiu ar sudėtingu užduotis sukurti daugybę darbo eigų iš skirtingų išankstinio apdorojimo rinkinių ir/arba modelio specifikacijų, o tada po vieną apskaičiuoti našumo rodiklius.

Pažiūrėkime, ar galime tai išspręsti sukurdami funkciją, kuri pritaiko darbo eigų sąrašą mokymo rinkiniui, o tada grąžina našumo rodiklius, remdamasi testavimo rinkiniu. Naudosime `map()` ir `map_dfr()` iš [purrr](https://purrr.tidyverse.org/) paketo, kad pritaikytume funkcijas kiekvienam sąrašo elementui.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) funkcijos leidžia pakeisti daugybę for ciklų kodu, kuris yra ir glaustesnis, ir lengviau skaitomas. Geriausia vieta sužinoti apie [`map()`](https://purrr.tidyverse.org/reference/map.html) funkcijas yra [iteracijos skyrius](http://r4ds.had.co.nz/iteration.html) knygoje "R for Data Science".


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) paketas leidžia vartotojams kurti ir lengvai pritaikyti daugybę modelių, tačiau jis daugiausia skirtas dirbti su persamplingo technikomis, tokiomis kaip `kryžminė validacija`, metodas, kurį dar aptarsime.

## **🚀Iššūkis**

Kiekviena iš šių technikų turi daugybę parametrų, kuriuos galite koreguoti, pavyzdžiui, `cost` SVM modeliuose, `neighbors` KNN modeliuose, `mtry` (atsitiktinai parinkti prediktoriai) Atsitiktinių Miškų modeliuose.

Ištyrinėkite kiekvieno modelio numatytuosius parametrus ir pagalvokite, ką šių parametrų koregavimas reikštų modelio kokybei.

Norėdami sužinoti daugiau apie konkretų modelį ir jo parametrus, naudokite: `help("model")`, pvz., `help("rand_forest")`.

> Praktikoje mes dažniausiai *įvertiname* *geriausias reikšmes* treniruodami daugybę modelių su `simuliuotu duomenų rinkiniu` ir matuodami, kaip gerai visi šie modeliai veikia. Šis procesas vadinamas **derinimu**.

### [**Po paskaitos testas**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Peržiūra ir savarankiškas mokymasis**

Šiose pamokose yra daug terminų, todėl skirkite minutę peržiūrėti [šį sąrašą](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) naudingų terminų!

#### DĖKOJAME:

[`Allison Horst`](https://twitter.com/allison_horst/) už nuostabius iliustracijas, kurios padaro R labiau prieinamą ir įdomų. Daugiau iliustracijų rasite jos [galerijoje](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) ir [Jen Looper](https://www.twitter.com/jenlooper) už originalios šio modulio Python versijos sukūrimą ♥️

Sėkmingo mokymosi,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambasadorius.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Piešinys @allison_horst</figcaption>



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
